In [10]:
from utils.__init__  import get_transaction_details

# Example usage
# txn_intent_hash = "txid_rdx1e7yx9ess0ulzpzhepdh9zaffcet0f70x93xdk7v9t83qrn738vfqc0es0w"  # Remove liquidity
txn_intent_hash = "txid_rdx12t6cs0n65wex7s9c52kpy4943xhx4uap0p9y6c08uf5m43gwch7skdk8ge"  # Add liquidity for c9 lsulp/xrd pool
txn_details = get_transaction_details(txn_intent_hash)

txn_details.json()
state_version = txn_details.json()['transaction']['state_version']   

# Call the function with the state_version we got earlier
from utils.__init__ import get_transaction_stream


response = get_transaction_stream(state_version)
parsed_response = response.json()
transaction_data = [x for x in parsed_response.get('items') if x.get('intent_hash') == txn_intent_hash]
transaction_data

[{'transaction_status': 'CommittedSuccess',
  'state_version': 258060362,
  'epoch': 191815,
  'round': 2263,
  'round_timestamp': '2025-04-04T17:13:25.017Z',
  'payload_hash': 'notarizedtransaction_rdx10ngvm998esac07dy54ww6s6gzt2ydqz04zx76dvtuvt0d2g6uauqy4jhn7',
  'intent_hash': 'txid_rdx12t6cs0n65wex7s9c52kpy4943xhx4uap0p9y6c08uf5m43gwch7skdk8ge',
  'fee_paid': '0.86787947537',
  'confirmed_at': '2025-04-04T17:13:25.017Z',
  'receipt': {'status': 'CommittedSuccess',
   'detailed_events': [{'identifier': {'package': 'package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx000538436477xxxxxxxxxresrce',
      'blueprint': 'FungibleVault',
      'event': 'LockFeeEvent'},
     'payload_type_definition': {'local_type_id': 22,
      'entity': 'package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx000538436477xxxxxxxxxresrce',
      'schema_hash_hex': '462a3fea283117aab2b01c297812bdc0fa9060b29eb5e68b847f361bc1201933',
      'kind': 'SchemaLocal'},
     'emitter': {'method_emitter': {'entity': 'internal_vault_rdx1tr70j03fkl

In [11]:
import pandas as pd


events = transaction_data[0]['receipt']['detailed_events']
# Extract data from events
events

# Initialize an empty list to store event data
event_data = []

# Process each event and extract relevant information
for event in events:
    # Initialize a dictionary for this event
    event_dict = {}
    
    # Extract identifier information
    if 'identifier' in event:
        event_dict['name'] = event['identifier'].get('event', '')
        event_dict['blueprint'] = event['identifier'].get('blueprint', '')
        event_dict['package'] = event['identifier'].get('package', '')
    
    # Extract emitter information
    if 'emitter' in event:
        event_dict['emitter_type'] = event['emitter'].get('type', '')
        
        # Handle different emitter structures
        if 'method_emitter' in event['emitter'] and 'entity' in event['emitter']['method_emitter']:
            entity = event['emitter']['method_emitter']['entity']
            if isinstance(entity, str):
                event_dict['entity_address'] = entity
            elif isinstance(entity, dict):
                event_dict['entity_address'] = entity.get('entity_address', '')
        
        if 'global_emitter' in event['emitter']:
            event_dict['global_emitter'] = event['emitter']['global_emitter']
    
    # Extract payload information
    if 'payload' in event and 'programmatic_json' in event['payload']:
        payload = event['payload']['programmatic_json']
        event_dict['data_kind'] = payload.get('kind', '')
        
        # Extract fields from payload based on structure
        if payload.get('kind') == 'Tuple' and 'fields' in payload:
            for field in payload['fields']:
                if 'field_name' in field and 'value' in field:
                    event_dict[field['field_name']] = field['value']
        
        # Handle Enum type payloads
        elif payload.get('kind') == 'Enum' and 'fields' in payload:
            event_dict['variant_name'] = payload.get('variant_name', '')
            
            for field in payload['fields']:
                if isinstance(field, dict):
                    if 'field_name' in field and 'value' in field:
                        event_dict[field['field_name']] = field['value']
                    elif 'type_name' in field and 'value' in field:
                        event_dict[field['type_name']] = field['value']
    
    # Add the event dictionary to our list
    event_data.append(event_dict)

# Create a pandas DataFrame from the event data
events_df = pd.DataFrame(event_data)

# Display the DataFrame
events_df

,name,blueprint,package,emitter_type,entity_address,global_emitter,data_kind,amount,variant_name,ResourceAddress,liquidity_receipt_id,amount_change_x,amount_change_y,amount_after_x,amount_after_y,price_after
0,LockFeeEvent,FungibleVault,package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx0005384...,EntityMethod,internal_vault_rdx1tr70j03fkllr0nlq6ghh8ha99gq...,account_rdx12xwrtgmq68wqng0d69qx2j627ld2dnfufd...,Tuple,1.0718896056885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WithdrawEvent,FungibleVault,package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx0005384...,EntityMethod,internal_vault_rdx1tzyzcd3v94wyr3ydz8g5pd34mkd...,account_rdx12xwrtgmq68wqng0d69qx2j627ld2dnfufd...,Tuple,77.163817016002229704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WithdrawEvent,Account,package_rdx1pkgxxxxxxxxxaccntxxxxxxxxxx0009296...,EntityMethod,account_rdx12xwrtgmq68wqng0d69qx2j627ld2dnfufd...,account_rdx12xwrtgmq68wqng0d69qx2j627ld2dnfufd...,Enum,NaN,Fungible,resource_rdx1thksg5ng70g9mmy9ne7wz0sc7auzrrwy7...,NaN,NaN,NaN,NaN,NaN,NaN
3,WithdrawEvent,FungibleVault,package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx0005384...,EntityMethod,internal_vault_rdx1tr70j03fkllr0nlq6ghh8ha99gq...,account_rdx12xwrtgmq68wqng0d69qx2j627ld2dnfufd...,Tuple,705.299964586673357414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WithdrawEvent,Account,package_rdx1pkgxxxxxxxxxaccntxxxxxxxxxx0009296...,EntityMethod,account_rdx12xwrtgmq68wqng0d69qx2j627ld2dnfufd...,account_rdx12xwrtgmq68wqng0d69qx2j627ld2dnfufd...,Enum,NaN,Fungible,resource_rdx1tknxxxxxxxxxradxrdxxxxxxxxx009923...,NaN,NaN,NaN,NaN,NaN,NaN
5,MintNonFungibleResourceEvent,NonFungibleResourceManager,package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx0005384...,EntityMethod,resource_rdx1ntrysy2sncpj6t6shjlgsfr55dns9290e...,resource_rdx1ntrysy2sncpj6t6shjlgsfr55dns9290e...,Tuple,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,MintLiquidityReceiptEvent,QuantaSwap,package_rdx1p4r9rkp0cq67wmlve544zgy0l45mswn6h7...,EntityMethod,component_rdx1crdhl7gel57erzgpdz3l3vr64scslq4z...,component_rdx1crdhl7gel57erzgpdz3l3vr64scslq4z...,Tuple,NaN,NaN,NaN,{0925f6a8bb2266c0-c725201c03711942-872b4c19f14...,NaN,NaN,NaN,NaN,NaN
7,DepositEvent,FungibleVault,package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx0005384...,EntityMethod,internal_vault_rdx1tqzs3ruz8npzcyqlvuvhkz4tkke...,component_rdx1crdhl7gel57erzgpdz3l3vr64scslq4z...,Tuple,77.163817016002229704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,DepositEvent,FungibleVault,package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx0005384...,EntityMethod,internal_vault_rdx1tradwury6373l0n2mc4al45lywr...,component_rdx1crdhl7gel57erzgpdz3l3vr64scslq4z...,Tuple,705.299964495764515997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AddLiquidityEvent,QuantaSwap,package_rdx1p4r9rkp0cq67wmlve544zgy0l45mswn6h7...,EntityMethod,component_rdx1crdhl7gel57erzgpdz3l3vr64scslq4z...,component_rdx1crdhl7gel57erzgpdz3l3vr64scslq4z...,Tuple,NaN,NaN,NaN,{0925f6a8bb2266c0-c725201c03711942-872b4c19f14...,77.163817016002229704,705.299964495764515997,NaN,NaN,NaN


In [12]:
# Filter the DataFrame for 'AddLiquidityEvent' and display 'amount_change_x' and 'amount_change_y'
events_df[['name','amount_change_x', 'amount_change_y']]


,name,amount_change_x,amount_change_y
0,LockFeeEvent,NaN,NaN
1,WithdrawEvent,NaN,NaN
2,WithdrawEvent,NaN,NaN
3,WithdrawEvent,NaN,NaN
4,WithdrawEvent,NaN,NaN
5,MintNonFungibleResourceEvent,NaN,NaN
6,MintLiquidityReceiptEvent,NaN,NaN
7,DepositEvent,NaN,NaN
8,DepositEvent,NaN,NaN
9,AddLiquidityEvent,77.163817016002229704,705.299964495764515997
